In [12]:
import pandas as pd
import xlsxloader as xl

df = xl.XlsxLoader.load_data()[0]

df = df.iloc[1: , :]
df = df.rename(columns = {"Chart": "Datetime", "Unnamed: 1": "Player count", "Unnamed: 2": "Average"})

print(df.shape)
print(df.isnull().sum()) # This provides a large number of missing values on second column as the average is only taken 

1 files found
(5990, 3)
              Datetime Player count Average
1  2011-11-30 00:00:00          680     NaN
2  2011-12-01 00:00:00          NaN     NaN
3  2011-12-02 00:00:00          NaN     NaN
4  2011-12-03 00:00:00          NaN     NaN
5  2011-12-04 00:00:00          NaN     NaN
Datetime           0
Player count     163
Average         4790
dtype: int64
